# Overview

This notebook demonstrates how to display Earth Engine generated tiles on an interactive map.

# Import Python modules

In [ ]:
import ee  # the Earth Engine Python API library
import ipyleaflet  # an interactive mapping "widget" for the notebook

# Initialize Earth Engine

The following command initializes the Earth Engine Python API. If the cell output displays an error about needing to authenticate, open up a  JupyterLab terminal tab and run the command: `earthengine authenticate`

In [ ]:
ee.Initialize()

# Display a Dynamic Map

To display an Earth Engine generated tiles on the interactive map, we will use ipyleaflet's TileLayer object. First we start by defining a function that can generate a tile layer URL from an Earth Engine image object.

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

Next we define a collection of imagery that we want to display. The following creates a collection of imagery from the Sentinel-2 satellite, filtered to a single day.

In [ ]:
s2 = ee.ImageCollection('COPERNICUS/S2').filterDate('2018-01-01', '2018-01-02')

Now we can create a map, add the Sentinel-2 tiles to it, and display it in the notebook.

In [ ]:
map1 = ipyleaflet.Map(
    center=(48.2082, 16.3779), zoom=4,
    layout={'height':'400px'}
)
map1.add_layer(
    ipyleaflet.TileLayer(url=GetTileLayerUrl(
        s2.mosaic().visualize(min=0, max=3000, gamma=1.5, bands= ['B4', 'B3', 'B2'])
    )
))
map1